In [1]:
from pyspark.sql.functions import explode, split, trim, lower, expr
from pyspark.sql import SparkSession, DataFrame
from pyspark import SparkConf
from pyspark import SparkContext
import logging
from os.path import abspath
from pathlib import Path
import shutil
from pathlib import Path
from typing import Optional, Union, List, Tuple, Any

In [2]:
print(sc._jvm.org.apache.hadoop.util.VersionInfo.getVersion())

3.3.4


In [3]:
print(spark._jsc.hadoopConfiguration().get("javax.jdo.option.ConnectionURL"))

jdbc:derby:;databaseName=/opt/spark/spark-warehouse/metastore_db;create=true


In [4]:
print(spark._jsc.hadoopConfiguration().get("hive.metastore.warehouse.dir"))

/opt/spark/spark-warehouse/


In [5]:
print(spark.conf.get("spark.sql.catalogImplementation"))

25/05/23 02:34:18 INFO  SharedState:60 Setting hive.metastore.warehouse.dir ('/opt/spark/spark-warehouse/') to the value of spark.sql.warehouse.dir.
25/05/23 02:34:18 WARN  MetricsConfig:136 Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
25/05/23 02:34:18 INFO  MetricsSystemImpl:378 Scheduled Metric snapshot period at 10 second(s).
25/05/23 02:34:18 INFO  MetricsSystemImpl:191 s3a-file-system metrics system started
25/05/23 02:34:19 WARN  VersionInfoUtils:85 The AWS SDK for Java 1.x entered maintenance mode starting July 31, 2024 and will reach end of support on December 31, 2025. For more information, see https://aws.amazon.com/blogs/developer/the-aws-sdk-for-java-1-x-is-in-maintenance-mode-effective-july-31-2024/
You can print where on the file system the AWS SDK for Java 1.x core runtime is located by setting the AWS_JAVA_V1_PRINT_LOCATION environment variable or aws.java.v1.printLocation system property to 'true'.
This messa

hive


In [6]:
spark.stop()

25/05/23 02:34:25 INFO  SparkContext:60 SparkContext is stopping with exitCode 0.
25/05/23 02:34:25 INFO  SparkUI:60 Stopped Spark web UI at http://db74469cb85a:4040
25/05/23 02:34:25 INFO  StandaloneSchedulerBackend:60 Shutting down all executors
25/05/23 02:34:25 INFO  StandaloneSchedulerBackend$StandaloneDriverEndpoint:60 Asking each executor to shut down
25/05/23 02:34:25 INFO  MapOutputTrackerMasterEndpoint:60 MapOutputTrackerMasterEndpoint stopped!
25/05/23 02:34:25 INFO  MemoryStore:60 MemoryStore cleared
25/05/23 02:34:25 INFO  BlockManager:60 BlockManager stopped
25/05/23 02:34:25 INFO  BlockManagerMaster:60 BlockManagerMaster stopped
25/05/23 02:34:25 INFO  OutputCommitCoordinator$OutputCommitCoordinatorEndpoint:60 OutputCommitCoordinator stopped!
25/05/23 02:34:25 INFO  SparkContext:60 Successfully stopped SparkContext


In [7]:
spark = (
    SparkSession.builder
    .appName("Init DB")
    .enableHiveSupport()
    .getOrCreate()
)

25/05/23 02:34:25 INFO  SparkContext:60 Running Spark version 3.5.5
25/05/23 02:34:25 INFO  SparkContext:60 OS info Linux, 5.15.146.1-microsoft-standard-WSL2, amd64
25/05/23 02:34:25 INFO  SparkContext:60 Java version 17.0.14
Setting Spark log level to "INFO".
25/05/23 02:34:25 INFO  ResourceUtils:60 ==============================================================
25/05/23 02:34:25 INFO  ResourceUtils:60 No custom resources configured for spark.driver.
25/05/23 02:34:25 INFO  ResourceUtils:60 ==============================================================
25/05/23 02:34:25 INFO  SparkContext:60 Submitted application: Init DB
25/05/23 02:34:25 INFO  ResourceProfile:60 Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 4, script: , vendor: , memory -> name: memory, amount: 2048, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
25/05/23 02:34:25 INFO  ResourceProfile:60 Limit

In [8]:
# Create dw schema in catalog
spark.sql("create database if not exists edw")
spark.sql("create database if not exists edw_stg")
spark.sql("create database if not exists edw_ld")
spark.sql("show databases").show()

25/05/23 02:34:26 INFO  SharedState:60 Setting hive.metastore.warehouse.dir ('/opt/spark/spark-warehouse/') to the value of spark.sql.warehouse.dir.
25/05/23 02:34:26 INFO  SharedState:60 Warehouse path is 's3a://delta-datawarehouse'.
25/05/23 02:34:31 INFO  HiveUtils:60 Initializing HiveMetastoreConnection version 2.3.9 using file:/opt/spark/jars/jersey-container-servlet-2.40.jar:file:/opt/spark/jars/netty-transport-native-kqueue-4.1.96.Final-osx-aarch_64.jar:file:/opt/spark/jars/parquet-common-1.13.1.jar:file:/opt/spark/jars/kubernetes-model-events-6.7.2.jar:file:/opt/spark/jars/jersey-client-2.40.jar:file:/opt/spark/jars/hive-shims-common-2.3.9.jar:file:/opt/spark/jars/py4j-0.10.9.7.jar:file:/opt/spark/jars/jackson-mapper-asl-1.9.13.jar:file:/opt/spark/jars/jaxb-runtime-2.3.2.jar:file:/opt/spark/jars/jakarta.annotation-api-1.3.5.jar:file:/opt/spark/jars/spark-mesos_2.12-3.5.5.jar:file:/opt/spark/jars/json-1.8.jar:file:/opt/spark/jars/netty-handler-4.1.96.Final.jar:file:/opt/spark/ja

AnalysisException: org.apache.hadoop.hive.ql.metadata.HiveException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient

In [ ]:
# Create Store Dim table
spark.sql("""drop table if exists edw.dim_store""")

spark.sql("""
create table edw.dim_store (
    row_wid string,
    store_id string,
    store_name string,
    address string,
    city string,
    state string,
    zip_code string,
    phone_number string,
    rundate string,
    insert_dt timestamp,
    update_dt timestamp
)
USING delta
""")

print("SPARK-APP: Store dimension created")

In [ ]:
# Create Plan Type Dimension
spark.sql("""drop table if exists edw.dim_plan_type""")
spark.sql("""
create table edw.dim_plan_type (
    plan_type_code string,
    plan_name string,
    rundate string,
    insert_dt timestamp,
    update_dt timestamp
)
USING delta
;
""")

print("SPARK-APP: Plan Type dimension created")

In [ ]:
# Create Date Dimension
spark.sql("""drop table if exists edw.dim_date""")
spark.sql("""
create table edw.dim_date (
    row_wid string,
    date date,
    day int,
    month int,
    year int,
    day_of_week string,
    rundate string,
    insert_dt timestamp,
    update_dt timestamp
)
USING delta
;
""")

print("SPARK-APP: Date dimension created")

In [ ]:
# Create Product Dimension
spark.sql("""drop table if exists edw.dim_product""")
spark.sql("""
create table edw.dim_product (
    row_wid string,
    product_id string,
    product_name string,
    brand string,
    type string,
    flavor string,
    size string,
    price double,
    expiration_dt date,
    image_url string,
    effective_start_dt timestamp,
    effective_end_dt timestamp,
    active_flg int,
    rundate string,
    insert_dt timestamp,
    update_dt timestamp
)
USING delta
;
""")

print("SPARK-APP: Product dimension created")

In [ ]:
# Create Customer Dimension
spark.sql("""drop table if exists edw.dim_customer""")
spark.sql("""
create table edw.dim_customer (
    row_wid string,
    customer_id string,
    first_name string,
    last_name string,
    address string,
    city string,
    state string,
    zip_code string,
    phone_number string,
    email string,
    date_of_birth date,
    plan_type string,
    effective_start_dt timestamp,
    effective_end_dt timestamp,
    active_flg int,
    rundate string,
    insert_dt timestamp,
    update_dt timestamp
)
USING delta
;
""")

print("SPARK-APP: Customer dimension created")

In [ ]:
# Create Sales Fact
spark.sql("""drop table if exists edw.fact_sales""")
spark.sql("""
create table edw.fact_sales (
    date_wid string,
    product_wid string,
    store_wid string,
    customer_wid string,
    order_id string,
    invoice_num string,
    qty int,
    tax double,
    discount double,
    line_total double,
    integration_key string,
    rundate string,
    insert_dt timestamp,
    update_dt timestamp
)
USING delta
;
""")

print("SPARK-APP: Sales Fact created")

In [ ]:
# Create Audit table
spark.sql("""drop table if exists edw.job_control""")
spark.sql("""
create table edw.job_control (
    schema_name string,
    table_name string,
    max_timestamp timestamp,
    rundate string,
    insert_dt timestamp
)
USING delta
;
""")

print("SPARK-APP: JOB Control table created")

In [ ]:
# Log all tables in Data Warehouse/Lakehouse

spark.sql("show tables in edw").show()

In [ ]:
spark.stop()